Análisis de sentimientos de mensajes en Amazon usando Naive Bayes
===

El archivo que se encuentra disponible en el link

https://raw.githubusercontent.com/jdvelasq/datalabs/master/datasets/amazon_cells_labelled.tsv

contiene mensajes escritos por los usuarios para productos comprados en Amazon y su valoración (positiva, negativa e indeterminada). En este laboratorio se debe construir un clasificador bayesiano que debe ser entrenado con los mensajes valorados, el cual debe ser posteriormente utilizado para valorar los mensajes con valoración indeterminada.

In [1]:
#
# Cargue el archivo usando Pandas e imprima la cantidad de
# registros
#
# Rta/
# 14609
#
import pandas as pd
import numpy as np

rows = pd.read_csv("https://raw.githubusercontent.com/jdvelasq/datalabs/master/datasets/amazon_cells_labelled.tsv", sep='\t', header=None)

rows.rename(columns={0:'message', 1:'qualification'}, inplace=True)

len(rows)

14609

In [2]:
#
# Imprima el primer mensaje de texto.
#
# Rta/
# 'I try not to adjust the volume setting to avoid that I turn off the call button which is situated just below the volume adjustment knob.'
#

rows['message'][0]

'I try not to adjust the volume setting to avoid that I turn off the call button which is situated just below the volume adjustment knob.'

In [3]:
#
# Imprima la cantidad de mensajes con NaN
#
# Rta/
# 13609
#

count_null = []

for q in range(len(rows['qualification'])):
    if pd.isna(rows['qualification'][q]) == True:
        count_null.append([rows['message'][q], rows['qualification'][q]])

len(count_null)

13609

In [4]:
#
# Imprima la cantidad de mensajes con valoración igual a 1.0
#
# Rta/
# 500
#
count_positive = []

for q in range(len(rows['qualification'])):
    if rows['qualification'][q] == 1.0:
        count_positive.append([rows['message'][q], rows['qualification'][q]])

len(count_positive)

500

In [5]:
#
# Imprima la cantidad de mensajes con valoración igual a 0.0
#
# Rta/
# 500
#

count_negative = []

for q in range(len(rows['qualification'])):
    if rows['qualification'][q] == 0.0:
        count_negative.append([rows['message'][q], rows['qualification'][q]])

len(count_negative)

500

In [6]:
#
# Genere un nuevo dataset que contenga únicamente los registros
# con valoración positiva o negativa e imprima su longitud
#
# Rta/
# 1000
#

new_rows = rows.copy()
new_rows = new_rows.dropna(subset=['qualification'])
len(new_rows)


1000

In [7]:
#
# Genere una nueva columna en el nuevo dataset computada como
# el resultado de aplicar el stemmer de Porter al mensaje e
# imprima el primer mensaje transformado
#
# Rta/
# 'so there is no way for me to plug it in here in the us unless i go by a converter.'
#

from nltk.stem import PorterStemmer

stemmer = PorterStemmer()

new_rows['message_stemmed'] = new_rows.message.apply(lambda x: " ".join([stemmer.stem(w) for w in x.split()]))

new_rows['message_stemmed'][1]

'so there is no way for me to plug it in here in the us unless i go by a converter.'

In [8]:
#
# Construya la matriz de terminos del documento considerando
# las palabras que tengan una frecuencia entre el 0.1% y el 98%,
# y que esten unicamente conformadas por letras.
#
# Imprima el tamaño del vocabulario.
#
# Rta/
# 1554
#
from sklearn.feature_extraction.text import CountVectorizer

# messages_splited = []

# for k in range(len(message_stemmed)):
#     message_stemmed[k] = str(message_stemmed[k])
#     messages_splited.append(message_stemmed[k].split(" "))

# new_rows['message_splited'] = messages_splited

count_vect = CountVectorizer(
    analyzer="word",                
    lowercase=True,                 
    stop_words="english",           
    #token_pattern=r"(?u)\b[a-zA-Z][a-zA-Z]+\b", 
    binary=True,                    
    max_df=0.98,                     
    min_df=0.001,                       
)

dtm = count_vect.fit_transform(new_rows.message_stemmed)

vocabulary = count_vect.get_feature_names()

len(vocabulary)


1554

In [9]:
#
# Construya un clasificador bayesiano que use los primeros
# 500 patrones para entrenamiento y los últimos 500 para
# prueba, e imprima el porcentaje de datos para cada clase
# para la muestra de entrenamiento-
#
# Rta/
# 1.0    52.2
# 0.0    47.8
# Name: label, dtype: float64
#

new_rows.rename(columns={'qualification':'label'}, inplace=True)

X_train = dtm[
    0:500,
]
X_test = dtm[
    500:,
]

y_train_true = new_rows.label[:500]
y_test_true = new_rows.label[500:]

data_train = round(100 * y_train_true.value_counts() / sum(y_train_true.value_counts()), 1)

data_train

1.0    52.2
0.0    47.8
Name: label, dtype: float64

In [10]:
#
# Imprima el porcentaje de datos para cada clase para la muestra
# de prueba, redondeado a un decimal.
#
# Rta/
# 0.0    52.2
# 1.0    47.8
# Name: label, dtype: float64
#

data_test = round(100 * y_test_true.value_counts() / sum(y_test_true.value_counts()), 1)
data_test

0.0    52.2
1.0    47.8
Name: label, dtype: float64

In [11]:
#
# Cree un clasificador de Bayes y entrenelo. Realice el pronostico
# para la muestra de entrenamiento y compute la matriz de confusion
#
# Rta/
# array([[214,  25],
#        [  1, 260]])
#
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import confusion_matrix

clf = BernoulliNB()

clf.fit(X_train, y_train_true)

y_test_pred = clf.predict(X_train)

cm_train = confusion_matrix(y_true=y_train_true, y_pred=y_test_pred)
cm_train.astype('int')

array([[214,  25],
       [  1, 260]])

In [14]:
#
# Realice el pronóstico para la muestra de entrenamiento y compute
# la matriz de confusión
#
# Rta/
# array([[153, 108],
#        [ 32, 207]])
#
y_test_pred = clf.predict(X_test)

cm_test = confusion_matrix(y_true=y_test_true, y_pred=y_test_pred)
cm_test.astype('int')

array([[153, 108],
       [ 32, 207]])

In [15]:
#
# Realice el pronostico para los mensajes con valoración 
# indeterminada y compute la cantidad de mensajes positivos
#
# Rta/
# 8284
#
import warnings
# init
warnings.filterwarnings("ignore")

rows_ = rows.copy()
rows_null = rows_[rows_.qualification.isnull()]

rows_null['message_stemmed'] = rows_null.message.apply(lambda x: " ".join([stemmer.stem(w) for w in x.split()])).str.lower()

rows_null

dtm_null = count_vect.transform(rows_null.message_stemmed)

y_pred_positive = (clf.predict(dtm_null))

int(sum(y_pred_positive))

8284